## Back Test Data Generation

In [1]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from pycaret.classification import *

In [2]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [3]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    k_number = cfg['knn']['k_number']
    metric = cfg['knn']['metric']
    algorithm = cfg['knn']['algorithm']
    feature_1 = cfg['knn']['feature_1']
    feature_2 = cfg['knn']['feature_2']
    feature_3 = cfg['knn']['feature_3']
    feature_7 = cfg['knn']['feature_7']
    feature_8 = cfg['knn']['feature_8']   
    volume = cfg['feature']['volume']
    volume_size = cfg['sample']['volume_size']
    sample_count = cfg['sample']['count']
    candles = cfg['recommendation']['candle_count']
    pair = cfg['currency']['pair']

In [4]:
print('K Number:',k_number)
print('Metric:', metric)
print('Algorithm:', algorithm)
print('Candle Volume Size:', volume_size)
print('Random Sample Count:', sample_count)
print('Future Candle Count:', candles)
print('Pair:', pair)

K Number: 5
Metric: euclidean
Algorithm: brute
Candle Volume Size: 5000
Random Sample Count: 4000
Future Candle Count: 7
Pair: USDCAD


In [5]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [6]:
def find_k_similar_candles(candle_id, dataset, k=k_number):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = metric, algorithm = algorithm) 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]][feature_1],
                            dataset.iloc[indices.flatten()[i]][feature_2],
                            dataset.iloc[indices.flatten()[i]][feature_3],
                            dataset.iloc[indices.flatten()[i]][feature_7],
                            dataset.iloc[indices.flatten()[i]][feature_8],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      feature_1,
                      feature_2,
                      feature_3,
                      feature_7,
                      feature_8,
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [7]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [8]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'SMA_20', 'F_SMA_5', 'F_SMA_10', 'F_SMA_20', 'O-H', 'O-L', 'O-C', 'H-L',
       'H-C', 'L-C', 'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5'],
      dtype='object')

In [9]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-10-13,21:00:00,21,286,1,Tuesday,2015-10-13T21:00:00.000000000Z,964,1.13796,1.13966,1.13776,1.13965,1.1387,1.1376,1.1358,0.00095,0.00205,0.00385,-0.00170,0.00020,-0.00169,0.00190,0.00001,-0.00189,1,-0.00169,0.00170,0.00189,0.00112,0.00017
1,2015-10-14,01:00:00,1,287,2,Wednesday,2015-10-14T01:00:00.000000000Z,1534,1.13968,1.14014,1.13834,1.13888,1.1389,1.1378,1.1364,-0.00002,0.00108,0.00248,-0.00046,0.00134,0.00080,0.00180,0.00126,-0.00054,0,0.00080,0.00046,0.00054,-0.00169,0.00112
2,2015-10-14,05:00:00,5,287,2,Wednesday,2015-10-14T05:00:00.000000000Z,5982,1.13892,1.14276,1.13892,1.14057,1.1392,1.1381,1.1370,0.00137,0.00247,0.00357,-0.00384,0.00000,-0.00165,0.00384,0.00219,-0.00165,1,-0.00165,0.00384,0.00165,0.00080,-0.00169
3,2015-10-14,09:00:00,9,287,2,Wednesday,2015-10-14T09:00:00.000000000Z,7209,1.14060,1.14460,1.13912,1.14425,1.1402,1.1389,1.1378,0.00405,0.00535,0.00645,-0.00400,0.00148,-0.00365,0.00548,0.00035,-0.00513,1,-0.00365,0.00400,0.00513,-0.00165,0.00080
4,2015-10-14,13:00:00,13,287,2,Wednesday,2015-10-14T13:00:00.000000000Z,7805,1.14428,1.14481,1.14084,1.14446,1.1416,1.1399,1.1384,0.00286,0.00456,0.00606,-0.00053,0.00344,-0.00018,0.00397,0.00035,-0.00362,1,-0.00018,0.00053,0.00362,-0.00365,-0.00165


## Selecting 1000 Random Candles

In [10]:
random_samples = data[data[volume] > volume_size].sample(n = sample_count)
Test_Candle = list(random_samples.index.values)

In [11]:
Test_Candle[0:10]

[7643, 7301, 8834, 2763, 835, 8133, 1745, 8248, 4367, 4476]

In [12]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-10-13,21:00:00,21,286,1,Tuesday,2015-10-13T21:00:00.000000000Z,964,1.13796,1.13966,1.13776,1.13965,1.1387,1.1376,1.1358,0.00095,0.00205,0.00385,-0.00170,0.00020,-0.00169,0.00190,0.00001,-0.00189,1,-0.00169,0.00170,0.00189,0.00112,0.00017
1,2015-10-14,01:00:00,1,287,2,Wednesday,2015-10-14T01:00:00.000000000Z,1534,1.13968,1.14014,1.13834,1.13888,1.1389,1.1378,1.1364,-0.00002,0.00108,0.00248,-0.00046,0.00134,0.00080,0.00180,0.00126,-0.00054,0,0.00080,0.00046,0.00054,-0.00169,0.00112
2,2015-10-14,05:00:00,5,287,2,Wednesday,2015-10-14T05:00:00.000000000Z,5982,1.13892,1.14276,1.13892,1.14057,1.1392,1.1381,1.1370,0.00137,0.00247,0.00357,-0.00384,0.00000,-0.00165,0.00384,0.00219,-0.00165,1,-0.00165,0.00384,0.00165,0.00080,-0.00169
3,2015-10-14,09:00:00,9,287,2,Wednesday,2015-10-14T09:00:00.000000000Z,7209,1.14060,1.14460,1.13912,1.14425,1.1402,1.1389,1.1378,0.00405,0.00535,0.00645,-0.00400,0.00148,-0.00365,0.00548,0.00035,-0.00513,1,-0.00365,0.00400,0.00513,-0.00165,0.00080
4,2015-10-14,13:00:00,13,287,2,Wednesday,2015-10-14T13:00:00.000000000Z,7805,1.14428,1.14481,1.14084,1.14446,1.1416,1.1399,1.1384,0.00286,0.00456,0.00606,-0.00053,0.00344,-0.00018,0.00397,0.00035,-0.00362,1,-0.00018,0.00053,0.00362,-0.00365,-0.00165


# <font color='red'>CANDLE LOOP</font>

In [13]:
#LR_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_LR_25Nov2021_EURUSD')
#ET_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_ET_25Nov2021_EURUSD')
KNN_FINAL_MODEL = load_model('FINAL_MODELS/USDCAD/09-01-2022_07-10_PM_knn_USDCAD')
GBC_FINAL_MODEL = load_model('FINAL_MODELS/USDCAD/08-01-2022_07-00_PM_gbc_USDCAD')
LIGHTGBM_FINAL_MODEL = load_model('FINAL_MODELS/USDCAD/08-01-2022_07-22_PM_lightgbm_USDCAD')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [14]:
%%time
print ('Today: ' + today)

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
#                              'Rec1':[],
#                              'Rec1_P':[],
#                              'Rec2':[],
#                              'Rec2_P':[],
#                              'Rec3':[],
#                              'Rec3_P':[],
#                              'LR_Label':[],
#                              'LR_Score':[],
#                              'ET_Label':[],
#                              'ET_Score':[],
                              'KNN_Label':[],
                              'KNN_Score':[],
                              'LIGHTGBM_Label':[],
                              'LIGHTGBM_Score':[],
                              'GBC_Label':[],
                              'GBC_Score':[],                              
                             })

for candle_no in Test_Candle:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+candles]
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market= 1

    else:
        Current_Market = 0

    data = pd.read_csv(filename)
    data = data[[feature_1,
                 feature_2,
                 feature_3,
                 feature_7,
                 feature_8,
                ]]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    recs = []
    for indice in indices[1:5]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+candles]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade = 'BUY'
            recs.append((r2_score(Y, y_pred)*100))
        else:
            Predicted_Trade = 'SELL'
            recs.append((r2_score(Y, y_pred)*100) * -1)
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])
        
        
    data_unseen = pd.DataFrame ({
        'Rec1_Score': [recs[0]],
        'Rec2_Score': [recs[1]],
        'Rec3_Score': [recs[2]],
        'Rec4_Score': [recs[3]],
    })
    
#    lr_prediction = predict_model(LR_FINAL_MODEL, data=data_unseen)
#    LR_Label = lr_prediction['Label']
#    LR_Score = lr_prediction['Score']
    
#    et_prediction = predict_model(ET_FINAL_MODEL, data=data_unseen)
#    ET_Label = et_prediction['Label']
#    ET_Score = et_prediction['Score']
    
    knn_prediction = predict_model(KNN_FINAL_MODEL, data=data_unseen)
    KNN_Label = knn_prediction['Label']
    KNN_Score = knn_prediction['Score']
    
    lightgbm_prediction = predict_model(LIGHTGBM_FINAL_MODEL, data=data_unseen)
    LIGHTGBM_Label = lightgbm_prediction['Label']
    LIGHTGBM_Score = lightgbm_prediction['Score']

    gbc_prediction = predict_model(GBC_FINAL_MODEL, data=data_unseen)
    GBC_Label = gbc_prediction['Label']
    GBC_Score = gbc_prediction['Score']    
    
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
#              'Rec1': predicted_output[0][0],
#              'Rec1_P': predicted_output[0][1],
#              'Rec2': predicted_output[1][0],
#              'Rec2_P': predicted_output[1][1],
#              'Rec3': predicted_output[2][0],
#              'Rec3_P': predicted_output[2][1],
#              'LR_Label': LR_Label[0],
#              'LR_Score': LR_Score[0],
#              'ET_Label': ET_Label[0],
#              'ET_Score': ET_Score[0],
              'KNN_Label': KNN_Label[0],
              'KNN_Score': KNN_Score[0],
              'LIGHTGBM_Label': LIGHTGBM_Label[0],
              'LIGHTGBM_Score': LIGHTGBM_Score[0],
              'GBC_Label': GBC_Label[0],
              'GBC_Score': GBC_Score[0],              
             }
    
    result_output = result_output.append(result, ignore_index = True)

Today: 09-01-2022_07-14_PM
CPU times: user 4h 28min 23s, sys: 4h 17min 39s, total: 8h 46min 2s
Wall time: 1h 6min 46s


In [15]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [16]:
result_output = result_output[result_output['Current_Market_Fit'] > 20]
result_output = result_output.reset_index()
del result_output['index']

In [17]:
result_output.shape

(2873, 9)

In [18]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [19]:
#result_output['LR_Prediction'] = result_output['Current_Market'] - result_output['LR_Label']
#result_output['ET_Prediction'] = result_output['Current_Market'] - result_output['ET_Label']
result_output['KNN_Prediction'] = result_output['Current_Market'] - result_output['KNN_Label']
result_output['LIGHTGBM_Prediction'] = result_output['Current_Market'] - result_output['LIGHTGBM_Label']
result_output['GBC_Prediction'] = result_output['Current_Market'] - result_output['GBC_Label']

In [20]:
result_output.head()

,Candle_No,Current_Market_Fit,Current_Market,KNN_Label,KNN_Score,LIGHTGBM_Label,LIGHTGBM_Score,GBC_Label,GBC_Score,KNN_Prediction,LIGHTGBM_Prediction,GBC_Prediction
0,7643.0,45.0,1.0,0.0,0.5953,1.0,0.7649,0.0,0.7343,1.0,0.0,1.0
1,8834.0,22.0,1.0,1.0,0.5117,0.0,0.8269,1.0,0.7450,0.0,1.0,0.0
2,2763.0,65.0,0.0,0.0,0.5124,0.0,0.9247,0.0,0.5285,0.0,0.0,0.0
3,835.0,54.0,1.0,1.0,0.9988,1.0,0.9096,1.0,0.9661,0.0,0.0,0.0
4,8133.0,57.0,0.0,0.0,0.9989,1.0,0.6912,0.0,0.9236,0.0,-1.0,0.0


In [21]:
LIGHTGBM_Result = pd.DataFrame(result_output['LIGHTGBM_Prediction'].value_counts())
LIGHTGBM_Result["Score"] = (LIGHTGBM_Result['LIGHTGBM_Prediction'] * 100 / len(result_output)).round(2)

In [22]:
KNN_Result = pd.DataFrame(result_output['KNN_Prediction'].value_counts())
KNN_Result["Score"] = (KNN_Result['KNN_Prediction'] * 100 / len(result_output)).round(2)

In [23]:
GBC_Result = pd.DataFrame(result_output['GBC_Prediction'].value_counts())
GBC_Result["Score"] = (GBC_Result['GBC_Prediction'] * 100 / len(result_output)).round(2)

In [24]:
result = pd.concat([
#    LR_Result,
#    ET_Result,
    KNN_Result,
    GBC_Result,
    LIGHTGBM_Result,
], axis=1)
result

,KNN_Prediction,Score,GBC_Prediction,Score,LIGHTGBM_Prediction,Score
0.0,2232,77.69,2188,76.16,2159,75.15
1.0,340,11.83,353,12.29,373,12.98
-1.0,301,10.48,332,11.56,341,11.87
